In [2]:
%runfile ../General-Tools/LazyList.sage
%runfile ../TS-Methods/MonoidTS.sage
%runfile ../TS-Methods/RingTS.sage
%runfile ../Sequence-Methods/Berlekamp.sage
%runfile ../TS-Methods/ComplexTS.sage

In [3]:
TS = ComplexTannakianSymbols()
ts = TS.parseSymbol

In [4]:
import pickle
import json

In [5]:
import random

maxEvenDim = 10
maxOddDim = 10
elementRange = (1, 5)

precision = 20

length = 50000

data = [None] * length
symbols = [None] * length

def printProgress(i, max):
    sys.stdout.write("\r%d (%d%%)" % (i, 100 * i / max))
    sys.stdout.flush()

In [7]:

random.seed(42)
for i in range(length):
    evendim = random.randint(0, maxEvenDim)
    odddim  = random.randint(0, maxOddDim)
    sym = []
    for j in range(evendim):
        sym.append([random.randint(*elementRange), 1])
    for j in range(odddim):
        sym.append([random.randint(*elementRange), -1])
    symbol = TS.createElement(sym)
    data[i] = (symbol.getBellCoefficients().toList(precision), symbol.evendimension())
    symbols[i] = symbol
    
    if i != 0 and (i % 10) == 0:
        printProgress(i, length)

printProgress(length, length)

50000 (100%)

In [8]:
with open('recursionData.json', 'w') as f:
    json.dump(map(lambda (l, r): (map(float, l), int(r)), data), f)

In [9]:
with open('recursionData.json') as f:
    data2 = map(lambda (l, r): (map(float, l), int(r)), json.load(f))

In [10]:
training_size = 45000

In [11]:
testing_size = 5000

In [12]:
properData = map(lambda (coeffs, evendim): (coeffs, [0]*(evendim) + [1] + [0]*(maxEvenDim - evendim)), data2)

In [13]:
training_data = properData[0:training_size]

In [14]:
testing_data = properData[training_size:training_size + testing_size]

In [84]:
import tensorflow as tf
import numpy as np

In [16]:
x = tf.placeholder(tf.float32, [None, precision])

In [65]:
W = tf.Variable(tf.zeros([precision, precision], tf.float32))
W1 = tf.Variable(tf.zeros([precision, precision], tf.float32))
W2 = tf.Variable(tf.zeros([precision, maxEvenDim + 1], tf.float32))
b = tf.Variable(tf.zeros([maxEvenDim + 1]))

In [67]:
y1 = tf.nn.softmax(tf.matmul(x, W))
y2 = tf.nn.softmax(tf.matmul(y1, W1))
y = tf.nn.softmax(tf.matmul(y2, W2) + b)

In [68]:
y_ = tf.placeholder(tf.float32, [None, maxEvenDim + 1])

In [69]:
# cross entropy might not be ideal
cost = -tf.reduce_mean(tf.reduce_sum(y * y_))

In [70]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

In [71]:
init = tf.global_variables_initializer()

In [72]:
sess = tf.Session()

In [73]:
sess.run(init)

In [82]:
printProgress(0, training_size)
for i, (bellCoeffs, evendim) in enumerate(training_data):
    sess.run(train_step, feed_dict={x: [bellCoeffs], y_: [evendim]})
    
    #if sess.run(tf.reduce_sum(tf.cast(tf.is_nan(W), tf.float32))) > 0:
        #print 
        #print "became nan at data[", i, "]"
        #break
        
    if (i + 1) % 50 == 0:
        printProgress(i + 1, training_size)

45000 (100%)

In [83]:
testing_data_input = map(lambda (bellCoeffs, evendim): bellCoeffs, testing_data)
testing_data_output = map(lambda (bellCoeffs, evendim): evendim, testing_data)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print sess.run(accuracy, feed_dict={x: testing_data_input, y_: testing_data_output})

0.2188


In [78]:
sess.close()

In [81]:
sess.run(y, feed_dict={x: [map(lambda n: n, range(20))]})

array([[  9.98439133e-01,   3.06447648e-04,   2.12071580e-04,
          2.01341609e-04,   1.51995249e-04,   1.47978964e-04,
          1.35576003e-04,   1.10373665e-04,   1.04805789e-04,
          9.64957362e-05,   9.37264413e-05]], dtype=float32)

In [38]:
training_data[16][0]

[1.0,
 30.0,
 526.0,
 7034.0,
 79408.0,
 797030.0,
 7333646.0,
 63093494.0,
 514482413.0,
 4015367920.0,
 30214508816.0,
 220429119004.0,
 1565994824568.0,
 10871832566260.0,
 73968056728836.0,
 494350479216564.0,
 3251817326757873.0,
 2.108799960766305e+16,
 1.3501234845233171e+17,
 8.544068549472751e+17]

In [39]:
sess.run(W)